#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

In [ ]:
DATASET_DISPLAY_NAME = "[your-dataset-display-name]
MODEL_DISPLAY_NAME = '"[your-model-display-name]
ENDPOINT_DISPLAY_NAME = '"[-endpoint-display-name]
#Set the target column representing the suitability or preference score of a stock
TARGET_COLUMN = 'preference_score'
# Set the BigQuery dataset and table names
DATASET_NAME = 'your-bigquery-dataset-name'
TABLE_NAME = 'your-bigquery-table-name'
# Set the Model Registry and Feature Store locations
MODEL_REGISTRY_LOCATION = 'us-central1'
FEATURE_STORE_LOCATION = 'us-central1'
# Set the staging bucket for Vertex AI
STAGING_BUCKET = 'gs://your-staging-bucket'


In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)
secret_manager_client = secretmanager.SecretManagerServiceClient()
# Custom component for AutoML Tabular training

aiplatform.init(project=project, location=location, staging_bucket=STAGING_BUCKET)
dataset = aiplatform.TabularDataset.get(display_name=dataset_display_name)
model = aiplatform.AutoMLTabularTrainingJob(
display_name=display_name,
optimization_prediction_type='regression',
)
model.run(dataset=dataset, target_column=target_column)
# Save the trained model to the Model Registry
model.save(model_display_name=display_name)


In [ ]:
# Custom component for BigQuery query
client = bigquery.Client(project=project_id, location=location)
query_job = client.query(query)
query_job.result()


In [ ]:
# Custom component for creating a BigQuery view
client = bigquery.Client(project=project_id, location=location)
query_job = client.query(query)
query_job.result()


In [ ]:
# Custom component for performing sentiment analysis
sentiment_score = ...  # Code to perform sentiment analysis on the input text
return sentiment_score


In [ ]:
#Custom component for storing the endpoint ID in Secret Manager
secret_name = 'vertex-ai-endpoint-id'
parent = secret_manager_client.secret_path(project_id, secret_name)
secret = secret_manager_client.create_secret(
request={
"parent": parent,
"secret_id": secret_name,
"secret": {"replication": {"automatic": {}}},
"}
)
secret_payload = endpoint_id.encode("UTF-8")
secret_manager_client.add_secret_version(
request={
"parent": parent,
"payload": {"data": secret_payload},
}
)


In [ ]:
#  Define the pipeline function
@dsl.pipeline(
name='Stock Recommendation Pipeline',
description='Pipeline for training an AutoML model and deploying an endpoint for stock recommendation.'
